In [1]:
import pandas as pd
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
from sklearn.pipeline import Pipeline

In [3]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
validation=pd.read_csv('validation.csv')

In [4]:
train.shape

(17877, 5)

In [5]:
test.shape,validation.shape

((3831, 5), (3831, 5))

In [6]:
train.head(5)

,review_id,title,year,user_review,user_suggestion
0,460,Black Squad,2018.0,"Early Access ReviewVery great shooter, that ha...",1
1,2166,Tree of Savior (English Ver.),2016.0,I love love love playing this game!Super 100%!...,1
2,17242,Eternal Card Game,2016.0,Early Access ReviewAs a fan of MTG and Hearths...,1
3,6959,Tactical Monsters Rumble Arena,2018.0,Turn based strategy game similiar to FF Tactic...,1
4,8807,Yu-Gi-Oh! Duel Links,2017.0,This game has an insanely huge download for be...,0


In [7]:
test.head(5)

,review_id,title,year,user_review,user_suggestion
0,12053,Infestation: The New Z,2016.0,Unbelievable that this rehash copy and paste t...,0
1,12536,SMITE®,2015.0,I can't recommened this game in its current st...,0
2,747,Heroes & Generals,2016.0,Early Access ReviewThis game is constantly evo...,0
3,3214,World of Warships,2018.0,I play this game because it scratches an itch....,0
4,4036,World of Guns: Gun Disassembly,2016.0,"Finally, a game for people like us to enjoy! P...",1


In [8]:
validation.head(5)

,review_id,title,year,user_review,user_suggestion
0,8604,Dungeon Defenders II,2015.0,Early Access Review* Ok Played the first DD lo...,1
1,20407,Minion Masters,2017.0,Product received for freeEarly Access ReviewSo...,1
2,636,Magic Duels,2018.0,Game is extremely unfun to play unless you wan...,0
3,10217,Robocraft,2016.0,Early Access ReviewThis used to be an amazing ...,0
4,9564,Realm of the Mad God,2014.0,"With stunning visuals, an immersive storyline,...",1


In [9]:
nlp=spacy.load('en_core_web_sm',disable='ner')

In [10]:
def preprossing(text):
  preprocess=[]
  for doc in nlp.pipe(text,n_process=-1):
    pre=[token.lemma_.lower() for token in doc if token.is_alpha and token.lemma_ not in nlp.Defaults.stop_words]
    pre=' '.join(pre)
    preprocess.append(pre)
  return preprocess



In [11]:
train['user_review']=preprossing(train['user_review'])


In [12]:
test['user_review']=preprossing(test['user_review'])
validation['user_review']=preprossing(validation['user_review'])



In [13]:
train['user_review']

,user_review
0,early access reviewvery great shooter original...
1,i love love love play lot class choose bound s...
2,early access reviewas fan mtg hearthstone fun ...
3,turn base strategy game similiar ff tactic day...
4,game insanely huge download phone game blast v...
...,...
17872,early access reviewan interesting game kill i ...
17873,early access spend minute queue match people a...
17874,product receive freeearly access reviewgame ne...
17875,day i completely entrance game i finally manag...


#one hot encoder

In [14]:
counter_vec=CountVectorizer(min_df=0.001,binary=True)

In [15]:
counter_vec_ohe=counter_vec.fit_transform(train['user_review'])

In [16]:
naivebayes=MultinomialNB()
naivebayes.fit(counter_vec_ohe,train['user_suggestion'])
naivebayes.score(counter_vec_ohe,train['user_suggestion'])

0.8452760530290317

In [17]:
counter_vec_val=counter_vec.transform(validation['user_review'])

In [18]:
naivebayes.fit(counter_vec_val,validation['user_suggestion'])
naivebayes.score(counter_vec_val,validation['user_suggestion'])

0.8861915948838424

# count vecterizer

In [19]:
counter_vecterizer=CountVectorizer(min_df=0.001)
counter_vecterizer_train=counter_vecterizer.fit_transform(train['user_review'])
navie=MultinomialNB()
navie.fit(counter_vecterizer_train,train['user_suggestion'])
navie.score(counter_vecterizer_train,train['user_suggestion'])


0.8388991441517033

In [20]:
counter_vecterizer_val=counter_vecterizer.transform(validation['user_review'])
navie.fit(counter_vecterizer_val,validation['user_suggestion'])
navie.score(counter_vecterizer_val,validation['user_suggestion'])

0.880187940485513

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfid=TfidfVectorizer(min_df=0.001)
tfid_trans=tfid.fit_transform(train['user_review'])
model=MultinomialNB()
model.fit(tfid_trans,train['user_suggestion'])
model.score(tfid_trans,train['user_suggestion'])

0.8414722828215024

In [22]:
tfid_trans_val=tfid.transform(validation['user_review'])
model.fit(tfid_trans_val,validation['user_suggestion'])
model.score(tfid_trans_val,validation['user_suggestion'])

0.8820151396502218

#tfidf with n-grams

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfid_ngram=TfidfVectorizer(min_df=0.001,ngram_range=(1,3))
tfid_ngram_trans=tfid_ngram.fit_transform(train['user_review'])
model_ngram=MultinomialNB()
model_ngram.fit(tfid_ngram_trans,train['user_suggestion'])
model_ngram.score(tfid_ngram_trans,train['user_suggestion'])

0.859204564524249

In [24]:
tfid_ngram_val=tfid_ngram.transform(validation['user_review'])
model_ngram.fit(tfid_ngram_val,validation['user_suggestion'])
model_ngram.score(tfid_ngram_val,validation['user_suggestion'])

0.8995040459410075

In [25]:
import spacy
nlp=spacy.load('en_core_web_sm')
def removing_nouns(df):
  Preprocessing_text=[]
  for doc in df['user_review']:
    pre=[token.text for token in nlp(doc) if token.pos_ not in ['NOUN','PROPN']]
    pre=' '.join(pre)
    Preprocessing_text.append(pre)
  return Preprocessing_text

In [26]:
train_noun=removing_nouns(train)


In [27]:
tfid=TfidfVectorizer(min_df=0.001,ngram_range=(1,3))
tfid_trans=tfid.fit_transform(train_noun)
model=MultinomialNB()
model.fit(tfid_trans,train['user_suggestion'])
model.score(tfid_trans,train['user_suggestion'])

0.8118811881188119

In [79]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import TensorDataset,DataLoader

In [80]:
tfid=TfidfVectorizer(min_df=0.001,ngram_range=(1,3))
tfid_trans=tfid.fit_transform(train['user_review'])
tfid_trans_val=tfid.transform(validation['user_review'])


In [81]:
tfid_trans=torch.tensor(tfid_trans.toarray(),dtype=torch.float32)
tfid_trans_val=torch.tensor(tfid_trans_val.toarray(),dtype=torch.float32)

In [82]:
y_train=torch.tensor(train['user_suggestion'],dtype=torch.float32)
y_val=torch.tensor(validation['user_suggestion'],dtype=torch.float32)

In [83]:
dataset=TensorDataset(tfid_trans,y_train)
dataset_val=TensorDataset(tfid_trans_val,y_val)
dataloader=DataLoader(dataset,batch_size=64,shuffle=True)
dataloader_val=DataLoader(dataset_val,batch_size=64)

In [85]:
class neuralnetworks(nn.Module):
    def __init__(self):
        super(neuralnetworks, self).__init__()
        self.fc1 = nn.Linear(tfid_trans.shape[1], 128)
        self.bn1 = nn.BatchNorm1d(128)  # Batch normalization layer
        self.dropout = nn.Dropout(0.6)
        self.fc2 = nn.Linear(128, 64)
        self.bn2=nn.BatchNorm1d(64)
        self.dropout = nn.Dropout(0.6)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.leaky_relu(self.bn1(self.fc1(x)),negative_slope=0.01)
        x = self.dropout(x)
        x = F.leaky_relu(self.bn2(self.fc2(x)),negative_slope=0.01)
        x = self.dropout(x)
        x = torch.sigmoid(self.fc3(x))
        return x

In [86]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=neuralnetworks().to(device)

In [87]:
from sklearn.metrics import f1_score,log_loss
from torch.optim import Adam
loss=nn.BCELoss()

In [97]:
def module(model, lr, optimizer, epoches):
    trainloss_list = []
    validationloss_list = []
    best_val_loss=float('inf')
    patiency=0
    max_patiency=10
    for epoch in range(epoches):
        model.train()
        train_loss = 0
        for xbatch1, ybatch1 in dataloader:
            xbatch1 = xbatch1.to(device)
            ybatch1 = ybatch1.to(device)
            predict1 = model(xbatch1)
            loss_value = loss(predict1.squeeze(), ybatch1)
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()
            train_loss += loss_value.item()
        avg_train_loss = train_loss / len(dataloader)
        trainloss_list.append(avg_train_loss)

        model.eval()
        validation_loss = 0
        for xbatch, ybatch in dataloader_val:
            xbatch = xbatch.to(device)
            ybatch = ybatch.to(device)
            with torch.no_grad():
                predict2 = model(xbatch)
                loss_value = loss(predict2.squeeze(), ybatch)
                validation_loss += loss_value.item()
        avg_validation_loss = validation_loss / len(dataloader_val)
        validationloss_list.append(avg_validation_loss)
        f1score_train = f1_score(ybatch1.cpu().numpy(), torch.round(predict1.squeeze().detach()).cpu().numpy())
        f1score_val = f1_score(ybatch.cpu().numpy(), torch.round(predict2.squeeze().detach()).cpu().numpy())

        # The following lines were indented incorrectly, causing the error.
        # They should be aligned with the 'for' loop above.
        if best_val_loss>avg_validation_loss:
            best_val_loss=avg_validation_loss
            patiency=0

        else:
            patiency+=1
        if patiency>max_patiency:
            print(f'early stopping {epoch+1}')
            break

        # print for specified epochs
        if (epoch+1)%10==0:
            print(f'Epoch:{epoch+1}/{epoches},trainloss:{avg_train_loss:.16f},testloss:{avg_validation_loss},f1score_train:{f1score_train:.16f},f1score_val:{f1score_val:.16f}')

In [99]:
lr=0.001
optimizer=optim.Adam(model.parameters(),lr=lr,weight_decay=0.001)
epoches=50
module(model,lr,optimizer,epoches)

Epoch:10/50,trainloss:0.1279654947708228,testloss:0.476226041217645,f1score_train:1.0000000000000000,f1score_val:0.8615384615384616
early stopping 19


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# New Section